# UC Berkeley Milling Dataset
> Reproduces the results from the UC Berkeley Milling Dataset.

Here, we will reproduce the results, figures, and tables from the experiment using the UC Berekely Milling Dataset. You can find dataset on the [NASA Prognostics Repository](https://www.nasa.gov/content/prognostics-center-of-excellence-data-set-repository).

A detailed description of the dataset is found in the [PyPHM example notebook](https://github.com/tvhahn/PyPHM/blob/master/notebooks/milling_example.ipynb) (which you can also [run on Google Colab](https://colab.research.google.com/github/tvhahn/PyPHM/blob/master/notebooks/milling_example.ipynb)).

First step to reproduce the results is to get the proper environment setup and download the data.

# Setup
Use PyPHM to quickly download and prepare the dataset.

**Only if** you are running in Google Colab, install PyPHM.

In [ ]:
# ONLY RUN IF YOU'RE USING GOOGLE COLAB
!pip install pyphm
!git clone https://github.com/tvhahn/tspipe

Import PyPHM, NumPy, pandas, and pathlib.

In [1]:
from pyphm.datasets.milling import MillingPrepMethodA
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path

%load_ext autoreload
%autoreload 2